In [25]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz

data = load_npz('data/assistments17_first_attempt/X-isicsctcwa.npz')

In [20]:
for key in data.keys():
    print(key, data[key].shape)

indices (5052979,)
indptr (447588,)
format ()
shape (2,)
data (5052979,)


In [27]:
dense_matrix = data.toarray()

In [28]:
dense_matrix

array([[0.0000000e+00, 1.1170000e+03, 1.0492530e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.1180000e+03, 1.0493020e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.1190000e+03, 1.0493060e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.7070000e+03, 1.7110000e+03, 5.2026408e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.7070000e+03, 1.7120000e+03, 5.2026420e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.7070000e+03, 8.3500000e+02, 5.2026498e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]])

In [29]:
df = pd.DataFrame(dense_matrix)

In [30]:
df

,0,1,2,3,4,5,6,7,8,9,...,3467,3468,3469,3470,3471,3472,3473,3474,3475,3476
0,0.0,1117.0,1049253.0,0.0,339.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1118.0,1049302.0,1.0,339.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1119.0,1049306.0,0.0,338.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1120.0,1049330.0,1.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1087.0,1049406.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447582,1707.0,1709.0,52026203.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447583,1707.0,1710.0,52026396.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447584,1707.0,1711.0,52026408.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447585,1707.0,1712.0,52026420.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
train_df = pd.read_csv(f'data/assistments17_first_attempt/preprocessed_data_train.csv', sep="\t")
test_df = pd.read_csv(f'data/assistments17_first_attempt/preprocessed_data_test.csv', sep="\t")

In [ ]:
    # Student-wise train-test split
    user_ids = X[:, 0].toarray().flatten()
    users_train = train_df["user_id"].unique()
    users_test = test_df["user_id"].unique()
    train = X[np.where(np.isin(user_ids, users_train))]
    test = X[np.where(np.isin(user_ids, users_test))]

In [34]:
# get column 1 of df and remove duplicates
user_ids = df.iloc[:, 0].drop_duplicates()

# get train and test user_id
users_train = train_df["user_id"].unique()
users_test = test_df["user_id"].unique()

In [35]:
# train df where user_id is in users_train
train = df[df.iloc[:, 0].isin(users_train)]

# test df where user_id is in users_test
test = df[df.iloc[:, 0].isin(users_test)]

MemoryError: Unable to allocate 9.25 GiB for an array with shape (357001, 3477) and data type float64

### Dummy curriculum mapping - knowledge components and Q-matrix

In [8]:
import pandas as pd
import numpy as np

In [1]:


sheet1 = pd.read_excel("/Users/harizliewzw/Documents/Adminstrative/Courses/Dummy101/DUMMY101 curriculum_mapping.xlsx", sheet_name=0)
sheet2 = pd.read_excel("/Users/harizliewzw/Documents//Adminstrative/Courses/Dummy101/DUMMY101 curriculum_mapping.xlsx", sheet_name=1) 

In [2]:
# Reverse the multi-hot encoding
def decode_multihot(row):
    return [col for col in sheet2.columns if row[col] == 1]

sheet1['knowledge component'] = sheet2.apply(decode_multihot, axis=1)

In [4]:
# Multi-hot encode the 'Decoded' column
encoded_df = pd.get_dummies(sheet1['knowledge component'].apply(pd.Series).stack()).sum(level=0)

print(encoded_df)

    addition  comparing  counting  division  money  multiplication  patterns  \
0          0          0         1         0      0               0         0   
1          0          0         1         0      0               0         0   
2          0          0         1         0      0               0         0   
3          0          0         0         0      0               0         1   
4          0          1         0         0      0               0         0   
5          1          0         0         0      0               0         0   
6          1          0         0         0      0               0         0   
7          0          0         0         0      0               0         0   
8          0          0         0         0      0               0         0   
9          1          0         0         0      0               0         0   
10         0          0         0         0      0               0         0   
12         0          0         0       

C:\Users\harizliewzw\AppData\Local\Temp\ipykernel_1432\3476277911.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  encoded_df = pd.get_dummies(sheet1['knowledge component'].apply(pd.Series).stack()).sum(level=0)
C:\Users\harizliewzw\AppData\Local\Temp\ipykernel_1432\3476277911.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  encoded_df = pd.get_dummies(sheet1['knowledge component'].apply(pd.Series).stack()).sum(level=0)


In [5]:
def get_weights(df):
    df2 = df.iloc[:, 1:]
    nodes = df2.columns
    arr = df2.values
    m = np.dot(arr.T, arr).astype(float)
    idx = np.tril_indices(m.shape[0])   
    m[idx] = np.nan
    result = pd.DataFrame(m, columns=nodes, index=nodes)
    result = result.stack()
    result = result.astype(int)
    result = result.reset_index()
    result.columns = ['Node A', 'Node B', 'Weights']
    return result

In [6]:
sheet2.head(1)

,Item ID,addition,substraction,multiplication,division,counting,money,comparing,patterns
0,101001,0,0,0,0,1,0,0,0


In [28]:
df = get_weights(sheet2)

In [33]:
df

,Node A,Node B,Weights
0,addition,substraction,3
1,addition,multiplication,1
2,addition,division,2
3,addition,counting,1
4,addition,money,3
5,addition,comparing,0
6,addition,patterns,0
7,substraction,multiplication,2
8,substraction,division,2
9,substraction,counting,1


In [30]:
from pyvis.network import Network

In [38]:
# Create a pyvis network
net = Network(notebook=True, filter_menu=True)

# Extract unique nodes from the DataFrame
unique_nodes = set(df['Node A']).union(set(df['Node B']))

# Add nodes to the network
for node in unique_nodes:
    net.add_node(node)

# Add edges with weights to the network
for index, row in df.iterrows():
    src, dest, weight = row['Node A'], row['Node B'], row['Weights']
    net.add_edge(src, dest, value=weight)
    
# Save the network graph as an HTML file
net.show_buttons()
net.show("network_graph.html")

# Open the HTML file in your web browser
#import webbrowser
# webbrowser.open("network_graph.html")

network_graph.html
